# 51. Introduction to Cumulative Aggregations and Ranking in SQL Queries

# 52. Overview of CTAS to create tables based on Query Results

In [1]:
%%sql
CREATE TABLE order_count_by_status
AS
SELECT orders.order_status, count(*) AS order_count
FROM orders
GROUP BY 1;

SELECT * FROM order_count_by_status;

CREATE TABLE orders_stg
AS
SELECT * FROM orders WHERE 1=2;

-- SELECT o.order_date,
--     round(sum(oi.order_item_subtotal)::numeric, 2) AS order_revenue
-- FROM orders AS o
--     JOIN order_items AS oi
--         ON o.order_id = oi.order_item_order_id
-- WHERE o.order_status IN ('COMPLETE', 'CLOSED')
-- GROUP BY 1

,order_status,order_count
0,COMPLETE,22899
1,ON_HOLD,3798
2,PENDING_PAYMENT,15030
3,PENDING,7610
4,CLOSED,7556
5,CANCELED,1428
6,PROCESSING,8275
7,PAYMENT_REVIEW,729
8,SUSPECTED_FRAUD,1558


In [2]:
%%sql
SELECT * FROM orders_stg;

,order_id,order_date,order_customer_id,order_status


In [3]:
%%sql
SELECT * FROM order_count_by_status;

,order_status,order_count
0,COMPLETE,22899
1,ON_HOLD,3798
2,PENDING_PAYMENT,15030
3,PENDING,7610
4,CLOSED,7556
5,CANCELED,1428
6,PROCESSING,8275
7,PAYMENT_REVIEW,729
8,SUSPECTED_FRAUD,1558


# 53. Create Tables for Cumulative Aggregations and Ranking